# Model Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

### Definitions

In [2]:
PATH_TO_SAVE_DATA = "."
PATH_DF_POS_FR = PATH_TO_SAVE_DATA + '/' + 'df_pos_fr.csv'
PATH_DF_TEST_FR = PATH_TO_SAVE_DATA + '/' + 'df_test_fr.csv'
PATH_JSON_METEO_FR = PATH_TO_SAVE_DATA + '/' + 'data_meteo_fr.json'
PATH_DF_FEAT_FR = PATH_TO_SAVE_DATA + '/' + 'df_feat_fr.csv' 
PATH_GEO_DEP_FR = PATH_TO_SAVE_DATA + '/sources/geofrance/' + 'departments.csv'
PATH_MDL_MULTI_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_multi_step_pos_fr_tcn"
PATH_MDL_MULTI_TFLITE = PATH_TO_SAVE_DATA + '/' + \
    'serverless/tensorflow_lite_on_aws_lambda'
PATH_MDL_MULTI_TFLITE_FILE = PATH_MDL_MULTI_TFLITE + '/' + \
    "converted_model.tflite"
PATH_SERVERLESS = PATH_MDL_MULTI_TFLITE + '/' + 'serverless.yml'

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140734 # on 13/05/2020
NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer'

# model 
PAST_HISTORY = 14 # days used to predict next values in future
FUTURE_TARGET = 7 # nb predict days later
STEP = 1

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")

TRAIN_SPLIT = 346


### helper functions

In [3]:
# save file before update
def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
       
        res_re = re.search('\.\w+$', path_file_name)
        
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
         
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))

### load data

In [4]:
df_feat_fr = pd.read_csv(PATH_DF_FEAT_FR)
df_feat_fr.index = df_feat_fr["date"]
df_feat_fr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_fr.shape[0])]
df_feat_fr


,T_min,date,T_max,H_min,H_max,extrap,pos,age_pos,test,age_test,day_num,nb_cases,sum_cases,Rt,rate_pos,train
date,,,,,,,,,,,,,,,,
2020-05-13,284.926667,2020-05-13,290.505000,64.661017,88.135593,0.0,884,61.085973,39096,55.449279,3,141111,NaN,NaN,2.261101,True
2020-05-14,285.050000,2020-05-14,290.963333,59.406780,84.847458,0.0,980,60.456122,42021,54.836820,4,142091,NaN,NaN,2.332167,True
2020-05-15,285.308333,2020-05-15,291.920000,57.372881,82.966102,0.0,1021,60.042116,47694,54.388833,5,143112,NaN,NaN,2.140730,True
2020-05-16,284.956667,2020-05-16,293.500000,53.741379,86.534483,0.0,292,60.085616,16541,54.488000,6,143404,NaN,NaN,1.765310,True
2020-05-17,285.598333,2020-05-17,294.446667,49.879310,85.500000,0.0,139,61.568345,6612,58.282970,0,143543,NaN,NaN,2.102238,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-03,281.508333,2021-05-03,291.938333,46.450000,86.866667,0.0,32488,42.164245,479742,43.617855,1,5626704,338361.0,0.682814,6.771973,False
2021-05-04,284.020000,2021-05-04,290.883333,55.266667,85.683333,0.0,22842,41.566632,409480,40.603736,2,5649546,326092.0,0.693442,5.578294,False
2021-05-05,284.170000,2021-05-05,290.341667,53.066667,86.883333,0.0,19969,40.927488,357256,43.091884,3,5669515,313036.0,0.682506,5.589549,False


### prepare features

In [5]:
features = df_feat_fr.copy().filter(items=['T_min', 'T_max', 'H_min',
                                           'H_max', 'pos', 'test', 'day_num',
                                          'age_pos', 'age_test'])
features

,T_min,T_max,H_min,H_max,pos,test,day_num,age_pos,age_test
date,,,,,,,,,
2020-05-13,284.926667,290.505000,64.661017,88.135593,884,39096,3,61.085973,55.449279
2020-05-14,285.050000,290.963333,59.406780,84.847458,980,42021,4,60.456122,54.836820
2020-05-15,285.308333,291.920000,57.372881,82.966102,1021,47694,5,60.042116,54.388833
2020-05-16,284.956667,293.500000,53.741379,86.534483,292,16541,6,60.085616,54.488000
2020-05-17,285.598333,294.446667,49.879310,85.500000,139,6612,0,61.568345,58.282970
...,...,...,...,...,...,...,...,...,...
2021-05-03,281.508333,291.938333,46.450000,86.866667,32488,479742,1,42.164245,43.617855
2021-05-04,284.020000,290.883333,55.266667,85.683333,22842,409480,2,41.566632,40.603736
2021-05-05,284.170000,290.341667,53.066667,86.883333,19969,357256,3,40.927488,43.091884


### Prepare dataset

In [6]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
dataset = (dataset-data_mean)/data_std

In [7]:
dataset.shape[1]

9

In [8]:
PAST_HISTORY

14

In [9]:
dataset.shape[1]

9

### Load model

In [10]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP)

/anaconda3/envs/coronavirusModel/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1045: UserWarning: tcn.tcn is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


CPU times: user 1.38 s, sys: 122 ms, total: 1.5 s
Wall time: 1.87 s


In [11]:
multi_step_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 14, 9)]           0         
_________________________________________________________________
tcn (TCN)                    (None, 64)                43136     
_________________________________________________________________
dense (Dense)                (None, 7)                 455       
Total params: 43,591
Trainable params: 43,591
Non-trainable params: 0
_________________________________________________________________


In [12]:
multi_step_model.inputs[0].dtype

tf.float32

In [13]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_tcn"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./keras_tcn/assets


### Save model TFlite

In [14]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE)

File ./serverless/tensorflow_lite_on_aws_lambda/converted_model_20210513_17_15_20.tflite moved!


In [15]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

In [16]:
open(PATH_MDL_MULTI_TFLITE_FILE, "wb").write(tflite_model)

196784

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [17]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[283 - 297]
[290 - 304]
[297 - 311]
[304 - 318]
[311 - 325]
[318 - 332]
[325 - 339]
[332 - 346]
[339 - 353]
9


array([[12625.002  ,   812.1279 , 32628.348  , 30411.668  , 25571.75   ,
        28717.842  , 29130.918  , 17504.062  ,  3867.2656 , 42829.355  ,
        31617.336  , 31264.445  , 35936.434  , 35733.87   , 21650.307  ,
          303.41504, 51001.734  , 38199.938  , 36159.258  , 41580.75   ,
        41411.027  , 25209.348  ,  4266.3516 , 55886.414  , 42917.016  ,
        42696.62   , 42241.223  , 40895.156  , 24291.615  ,  1556.626  ,
        32905.344  , 54395.652  , 42239.504  , 40153.34   , 42065.105  ,
        22045.387  ,  4050.6064 , 38742.77   , 36152.72   , 34210.3    ,
        35270.676  , 33466.84   , 21846.68   ,  4346.3965 , 44620.617  ,
        33530.043  , 29047.668  , 29611.496  , 30708.117  , 20941.64   ,
        20108.002  , 33029.605  , 34652.887  , 24913.744  , 25485.781  ,
        23569.66   , 14181.089  , 12421.854  , 30799.14   , 20832.086  ,
        31349.742  , 43245.273  , 26669.346  ]], dtype=float32)

#### Future days

In [18]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [19]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [20]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE)

In [21]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

### Predict reloaded model

#### Past days

In [22]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [23]:
x_multi.shape

(1, 14, 9)

In [24]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [25]:
dataset.shape

(360, 9)

In [26]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[283 - 297]
[290 - 304]
[297 - 311]
[304 - 318]
[311 - 325]
[318 - 332]
[325 - 339]
[332 - 346]
[339 - 353]


In [27]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [28]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [29]:
y_multi_pred

array([[-0.16877234, -0.92532384,  1.1123352 ,  0.9703686 ,  0.66039777,
         0.86188805,  0.88834333,  0.1437054 , -0.7296586 ,  1.7656552 ,
         1.0475851 ,  1.0249845 ,  1.3242004 ,  1.3112273 ,  0.40925017,
        -0.9579042 ,  2.2890525 ,  1.4691657 ,  1.3384709 ,  1.6856886 ,
         1.6748189 ,  0.63718784, -0.7040993 ,  2.60189   ,  1.7712693 ,
         1.7571543 ,  1.7279884 ,  1.6417801 ,  0.5784119 , -0.87764275,
         1.1300752 ,  2.5064147 ,  1.7278783 ,  1.5942706 ,  1.716709  ,
         0.4345529 , -0.7179166 ,  1.503931  ,  1.3380523 ,  1.2136508 ,
         1.2815621 ,  1.166036  ,  0.4218269 , -0.6989728 ,  1.880376  ,
         1.1700838 ,  0.8830117 ,  0.91912186,  0.98935455,  0.36386386,
         0.3104738 ,  1.1380334 ,  1.241996  ,  0.6182559 ,  0.65489185,
         0.53217447, -0.06911331, -0.18178293,  0.9951841 ,  0.35684747,
         1.0304472 ,  1.7922924 ,  0.73069286]], dtype=float32)

In [30]:
y_multi_pred_out

array([[-0.16877255, -0.92532372,  1.1123358 ,  0.97036874,  0.66039789,
         0.86188817,  0.88834357,  0.14370513, -0.72965896,  1.76565588,
         1.04758561,  1.02498424,  1.32420039,  1.3112272 ,  0.40924993,
        -0.95790362,  2.28905177,  1.46916568,  1.33847058,  1.68568873,
         1.67481875,  0.6371879 , -0.7040987 ,  2.60189033,  1.7712692 ,
         1.75715411,  1.72798848,  1.64177978,  0.57841223, -0.87764359,
         1.13007498,  2.50641418,  1.72787833,  1.59427071,  1.71670926,
         0.43455303, -0.71791697,  1.50393128,  1.33805215,  1.21365118,
         1.28156209,  1.16603565,  0.42182678, -0.69897294,  1.88037634,
         1.17008328,  0.8830111 ,  0.91912162,  0.98935461,  0.36386383,
         0.31047371,  1.13803351,  1.24199617,  0.61825597,  0.65489185,
         0.53217465, -0.06911356, -0.18178278,  0.99518424,  0.35684776,
         1.03044689,  1.79229236,  0.73069286]])

In [31]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [32]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [33]:
y_future_pred_out

array([[-0.42999434, -0.84252596,  0.94284254,  0.33344281,  1.0169729 ,
         1.8877598 ,  0.5677042 ]])

In [34]:
y_future_pred

array([[-0.42999446, -0.8425262 ,  0.94284266,  0.33344257,  1.0169729 ,
         1.8877599 ,  0.56770414]], dtype=float32)

In [35]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda
    
- Execute : sls deploy -v

In [36]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd ./serverless/tensorflow_lite_on_aws_lambda\nserverless deploy -v'

In [37]:
open('deploy_serverless.sh', "w").write(str_exe)
os.chmod('deploy_serverless.sh', stat.S_IRWXU)

In [38]:
!cat ./deploy_serverless.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd ./serverless/tensorflow_lite_on_aws_lambda
serverless deploy -v

In [39]:
!./deploy_serverless.sh

Serverless: Deprecation warning: Support for "package.include" and "package.exclude" will be removed with next major release. Please use "package.patterns" instead
            More Info: https://www.serverless.com/framework/docs/deprecations/#NEW_PACKAGE_PATTERNS
Serverless: Deprecation warning: Resolution of lambda version hashes was improved with better algorithm, which will be used in next major release.
            Switch to it now by setting "provider.lambdaHashingVersion" to "20201221"
            More Info: https://www.serverless.com/framework/docs/deprecations/#LAMBDA_HASHING_VERSION_V2
Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/.serverless/requirements.txt...
Serverless: Using static c

### API AWS real Test

#### Past days

In [40]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[283 - 297]
[290 - 304]
[297 - 311]
[304 - 318]
[311 - 325]
[318 - 332]
[325 - 339]
[332 - 346]
[339 - 353]
status code :  200
[[[-0.16877242922782898, -0.9253236055374146, 1.1123356819152832, 0.9703688621520996, 0.6603978872299194, 0.8618881702423096, 0.8883435726165771]], [[0.14370518922805786, -0.7296589612960815, 1.7656558752059937, 1.0475856065750122, 1.024984359741211, 1.3242002725601196, 1.3112274408340454]], [[0.4092499613761902, -0.9579036235809326, 2.2890515327453613, 1.4691659212112427, 1.3384703397750854, 1.6856886148452759, 1.674818754196167]], [[0.6371880173683167, -0.7040989398956299, 2.6018903255462646, 1.7712692022323608, 1.7571537494659424, 1.7279884815216064, 1.6417797803878784]], [[0.5784120559692383, -0.8776435852050781, 1.1300749778747559, 2.5064141750335693, 1.727878451347351, 1.5942707061767578, 1.7167092561721802]], [[0.43455296754837036, -0.7179169654846191, 1.5039314031600952, 1.3380520343780518, 1.2136512994766235, 1.281562089920044, 1.1660356521606445]], [[

In [41]:
len(json_list_list_x)

23832

In [42]:
resp.json()

[[[-0.16877242922782898,
   -0.9253236055374146,
   1.1123356819152832,
   0.9703688621520996,
   0.6603978872299194,
   0.8618881702423096,
   0.8883435726165771]],
 [[0.14370518922805786,
   -0.7296589612960815,
   1.7656558752059937,
   1.0475856065750122,
   1.024984359741211,
   1.3242002725601196,
   1.3112274408340454]],
 [[0.4092499613761902,
   -0.9579036235809326,
   2.2890515327453613,
   1.4691659212112427,
   1.3384703397750854,
   1.6856886148452759,
   1.674818754196167]],
 [[0.6371880173683167,
   -0.7040989398956299,
   2.6018903255462646,
   1.7712692022323608,
   1.7571537494659424,
   1.7279884815216064,
   1.6417797803878784]],
 [[0.5784120559692383,
   -0.8776435852050781,
   1.1300749778747559,
   2.5064141750335693,
   1.727878451347351,
   1.5942707061767578,
   1.7167092561721802]],
 [[0.43455296754837036,
   -0.7179169654846191,
   1.5039314031600952,
   1.3380520343780518,
   1.2136512994766235,
   1.281562089920044,
   1.1660356521606445]],
 [[0.42182666063

In [43]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [44]:
y_multi_pred_out

array([[-0.16877243, -0.92532361,  1.11233568,  0.97036886,  0.66039789,
         0.86188817,  0.88834357,  0.14370519, -0.72965896,  1.76565588,
         1.04758561,  1.02498436,  1.32420027,  1.31122744,  0.40924996,
        -0.95790362,  2.28905153,  1.46916592,  1.33847034,  1.68568861,
         1.67481875,  0.63718802, -0.70409894,  2.60189033,  1.7712692 ,
         1.75715375,  1.72798848,  1.64177978,  0.57841206, -0.87764359,
         1.13007498,  2.50641418,  1.72787845,  1.59427071,  1.71670926,
         0.43455297, -0.71791697,  1.5039314 ,  1.33805203,  1.2136513 ,
         1.28156209,  1.16603565,  0.42182666, -0.69897306,  1.8803761 ,
         1.17008352,  0.88301098,  0.91912162,  0.98935455,  0.36386383,
         0.31047377,  1.13803363,  1.24199629,  0.61825603,  0.65489161,
         0.53217459, -0.06911341, -0.18178278,  0.99518424,  0.35684773,
         1.03044689,  1.79229236,  0.73069316]])

In [45]:
y_multi_pred

array([[-0.16877234, -0.92532384,  1.1123352 ,  0.9703686 ,  0.66039777,
         0.86188805,  0.88834333,  0.1437054 , -0.7296586 ,  1.7656552 ,
         1.0475851 ,  1.0249845 ,  1.3242004 ,  1.3112273 ,  0.40925017,
        -0.9579042 ,  2.2890525 ,  1.4691657 ,  1.3384709 ,  1.6856886 ,
         1.6748189 ,  0.63718784, -0.7040993 ,  2.60189   ,  1.7712693 ,
         1.7571543 ,  1.7279884 ,  1.6417801 ,  0.5784119 , -0.87764275,
         1.1300752 ,  2.5064147 ,  1.7278783 ,  1.5942706 ,  1.716709  ,
         0.4345529 , -0.7179166 ,  1.503931  ,  1.3380523 ,  1.2136508 ,
         1.2815621 ,  1.166036  ,  0.4218269 , -0.6989728 ,  1.880376  ,
         1.1700838 ,  0.8830117 ,  0.91912186,  0.98935455,  0.36386386,
         0.3104738 ,  1.1380334 ,  1.241996  ,  0.6182559 ,  0.65489185,
         0.53217447, -0.06911331, -0.18178293,  0.9951841 ,  0.35684747,
         1.0304472 ,  1.7922924 ,  0.73069286]], dtype=float32)

In [46]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [47]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
# REQUEST URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer' 
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[-0.4299943447113037, -0.8425259590148926, 0.9428426027297974, 0.3334426283836365, 1.0169728994369507, 1.8877599239349365, 0.5677042603492737]]]


In [48]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[-0.42999434, -0.84252596,  0.9428426 ,  0.33344263,  1.0169729 ,
         1.88775992,  0.56770426]])

In [49]:
y_future_pred

array([[-0.42999446, -0.8425262 ,  0.94284266,  0.33344257,  1.0169729 ,
         1.8877599 ,  0.56770414]], dtype=float32)

In [50]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
